In [ ]:
import os
import requests
import time
import json
from typing import Dict, Tuple, Optional

# LeetCode session information (replace with your own cookies/tokens)
LEETCODE_SESSION = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJuZXh0X2FmdGVyX29hdXRoIjoiLyIsIl9hdXRoX3VzZXJfaWQiOiI4NDAyNzg3IiwiX2F1dGhfdXNlcl9iYWNrZW5kIjoiZGphbmdvLmNvbnRyaWIuYXV0aC5iYWNrZW5kcy5Nb2RlbEJhY2tlbmQiLCJfYXV0aF91c2VyX2hhc2giOiIwMzJhYzg4OGIwMTVhYjc1MTU1ZTA2Y2FlNmYxMDMyNDM2ZDlhYmI1ZDRiZTllNzg5YTYyYTg4NTJiY2FiN2U1IiwiaWQiOjg0MDI3ODcsImVtYWlsIjoiIiwidXNlcm5hbWUiOiIxMTEtZWluIiwidXNlcl9zbHVnIjoiMTExLWVpbiIsImF2YXRhciI6Imh0dHBzOi8vYXNzZXRzLmxlZXRjb2RlLmNuL2FsaXl1bi1sYy11cGxvYWQvdXNlcnMvYXI5WTh0UmtmMy9hdmF0YXJfMTc1NzIyNTU0Ny5wbmciLCJwaG9uZV92ZXJpZmllZCI6dHJ1ZSwiaXAiOiIxNC4xMjcuNC4zOCIsIl90aW1lc3RhbXAiOjE3NTc5NDQzMjkuMDA0NDM3LCJleHBpcmVkX3RpbWVfIjoxNzYwNDY4NDAwLCJ2ZXJzaW9uX2tleV8iOjAsImRldmljZV9pZCI6IjA2Mjg3ZmQ2YzI2M2ZkNGMwNjFlMTllNzVjNGMwZDZjIn0.ZFggpvO7upjkLuE2tim-elHypIvze3oQTCAsPeLqKN8"
LEETCODE_CSRF = "nJzoIBX65k9b0oCvDx4vngnFJyjWJHE0lcwGyDA8fA3HrLZpTMxqd0X3PtoGJKHA"


class LeetCodeSubmitter:
    def __init__(self, solutions_dir: str, record_prefix: str = "submission_record"):
        """
        
        Initialize LeetCode submitter for C++ solutions.

        Args:
            solutions_dir (str): Directory containing C++ solution files.
            record_prefix (str): Prefix for submission record files.
        """
        self.solutions_dir = solutions_dir
        self.record_prefix = record_prefix
        self.session = requests.Session()

        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
            "Content-Type": "application/json",
            "X-CSRFToken": LEETCODE_CSRF,
            "Origin": "https://leetcode.cn",
            "Referer": "https://leetcode.cn/problems/",
        }

        self.session.cookies.set("LEETCODE_SESSION", LEETCODE_SESSION)
        self.session.cookies.set("csrftoken", LEETCODE_CSRF)

        self.total_problems = 0
        self.submitted_problems = 0
        self.accepted_problems = 0
        self.skipped_problems = 0

        self.record_index = 0
        self.record_file = f"{self.record_prefix}_{self.record_index}.cpp.txt"
        self.submission_records = self._load_submission_records()

        print("LeetCode session initialized. Ready to submit C++ solutions...")

    def _rotate_record_file(self):
        """Rotate record file if it exceeds 5MB."""
        if (
            os.path.exists(self.record_file)
            and os.path.getsize(self.record_file) > 5 * 1024 * 1024
        ):
            self.record_index += 1
            self.record_file = f"{self.record_prefix}_{self.record_index}.cpp.txt"
            print(f"Rotating record file -> {self.record_file}")

    def _load_submission_records(self) -> Dict[str, str]:
        """Load old submission records from previous runs."""
        records = {}
        for idx in range(1000):
            fname = f"{self.record_prefix}_{idx}.cpp.txt"
            if not os.path.exists(fname):
                break
            with open(fname, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line and ":" in line:
                        problem_name, status = line.split(":", 1)
                        records[problem_name.strip()] = status.strip()
        print(f"Loaded {len(records)} submission records from existing files.")
        return records

    def _save_submission_record(self, problem_name: str, status: str) -> None:
        """Save submission result to record file."""
        try:
            self._rotate_record_file()
            with open(self.record_file, "a", encoding="utf-8") as f:
                f.write(f"{problem_name}: {status}\n")
        except Exception as e:
            print(f"Error while saving submission record: {str(e)}")

    def _get_problem_slug(self, filename: str) -> str:
        basename = os.path.basename(filename)
        name_without_ext = os.path.splitext(basename)[0]
        if "." in name_without_ext:
            return name_without_ext.split(".", 1)[1]
        elif "-" in name_without_ext:
            return name_without_ext.split("-", 1)[1]
        return name_without_ext

    def _get_problem_id(self, slug: str) -> Optional[str]:
        graphql_url = "https://leetcode.cn/graphql/"
        graphql_payload = {
            "operationName": "questionData",
            "variables": {"titleSlug": slug},
            "query": """
                query questionData($titleSlug: String!) {
                    question(titleSlug: $titleSlug) {
                        questionId
                    }
                }
            """,
        }
        try:
            response = self.session.post(
                graphql_url, headers=self.headers, json=graphql_payload, timeout=10
            )
            if response.status_code == 200:
                data = response.json()
                q = data.get("data", {}).get("question", {})
                return q.get("questionId")
        except Exception as e:
            print(f"Error while getting problem ID: {str(e)}")
        return None

    def _submit_solution(self, problem_slug: str, solution_code: str) -> Tuple[str, str]:
        """Submit C++ code and return (status, runtime)."""
        problem_id = self._get_problem_id(problem_slug)
        if not problem_id:
            return "Failed", "Problem ID not found"

        submit_url = f"https://leetcode.cn/problems/{problem_slug}/submit/"
        payload = {"lang": "cpp", "question_id": problem_id, "typed_code": solution_code}

        try:
            response = self.session.post(
                submit_url, headers=self.headers, json=payload, timeout=30
            )
            if response.status_code == 200 and "submission_id" in response.json():
                sid = response.json()["submission_id"]
                print(f"Submitted successfully, submission ID: {sid}")
                return self._check_submission_result(sid)
            else:
                return "Failed", f"HTTP {response.status_code}"
        except Exception as e:
            return "Failed", str(e)

    def _check_submission_result(self, submission_id: str) -> Tuple[str, str]:
        """Check submission result."""
        for _ in range(30):  # max 30 retries
            try:
                check_url = f"https://leetcode.cn/submissions/detail/{submission_id}/check/"
                response = self.session.get(check_url, headers=self.headers, timeout=10)
                if response.status_code != 200:
                    return "Pending", f"HTTP {response.status_code}"
                result = response.json()
                if result.get("state") == "SUCCESS":
                    return (
                        result.get("status_msg", "UNKNOWN"),
                        result.get("status_runtime", "N/A"),
                    )
                time.sleep(1)
            except Exception as e:
                print(f"Error while checking result: {str(e)}")
                time.sleep(2)
        return "Timeout", "Max checks reached"

    def submit_all_solutions(self):
        solution_files = [f for f in os.listdir(self.solutions_dir) if f.endswith(".cpp")]
        solution_files.sort()
        self.total_problems = len(solution_files)
        print(f"Found {self.total_problems} C++ solutions.")

        for i, filename in enumerate(solution_files):
            problem_name = os.path.splitext(filename)[0]
            problem_slug = self._get_problem_slug(filename)
            solution_path = os.path.join(self.solutions_dir, filename)

            print(f"\n{'='*50}\nProcessing ({i+1}/{self.total_problems}): {problem_slug}")

            if problem_name in self.submission_records:
                print(f"Already submitted: {self.submission_records[problem_name]}, skipping.")
                self.skipped_problems += 1
                continue

            try:
                with open(solution_path, "r", encoding="utf-8") as f:
                    solution_code = f.read()

                status, runtime = self._submit_solution(problem_slug, solution_code)
                self._save_submission_record(problem_name, status)
                self.submission_records[problem_name] = status

                self.submitted_problems += 1
                if status == "Accepted":
                    self.accepted_problems += 1

                print(f"Result: {status}, Runtime: {runtime}")
                time.sleep(5)
            except Exception as e:
                print(f"Error while processing {filename}: {str(e)}")

        self.print_statistics()

    def print_statistics(self):
        print("\n" + "=" * 50)
        print("LeetCode C++ Solutions Submission Report")
        print("=" * 50)
        print(f"Total problems: {self.total_problems}")
        print(f"Submitted: {self.submitted_problems}")
        print(f"Skipped: {self.skipped_problems}")
        print(f"Accepted: {self.accepted_problems}")
        print(f"Failed: {self.submitted_problems - self.accepted_problems}")


def main():
    solutions_dir = "cpp_solutions"  # directory containing your .cpp files
    submitter = LeetCodeSubmitter(solutions_dir)
    submitter.submit_all_solutions()


if __name__ == "__main__":
    main()


Loaded 3009 submission records from existing files.
LeetCode session initialized. Ready to submit C++ solutions...
Found 3010 C++ solutions.

Processing (1/3010): two-sum
Already submitted: Accepted, skipping.

Processing (2/3010): regular-expression-matching
Already submitted: Accepted, skipping.

Processing (3/3010): same-tree
Already submitted: Accepted, skipping.

Processing (4/3010): minimum-cost-to-merge-stones
Already submitted: Accepted, skipping.

Processing (5/3010): grid-illumination
Already submitted: Accepted, skipping.

Processing (6/3010): find-common-characters
Already submitted: Accepted, skipping.

Processing (7/3010): check-if-word-is-valid-after-substitutions
Already submitted: Accepted, skipping.

Processing (8/3010): max-consecutive-ones-iii
Already submitted: Accepted, skipping.

Processing (9/3010): maximize-sum-of-array-after-k-negations
Already submitted: Accepted, skipping.

Processing (10/3010): clumsy-factorial
Already submitted: Accepted, skipping.

Proces